In [1]:
# Dependencies
import requests
import pandas as pd
from pprint import pprint
from config import api_key
import numpy as np
from IPython.display import Image, display
output_data_file = "movies_api.csv"
from sqlalchemy import create_engine, inspect

In [2]:
SQL_USERNAME = "postgres"
SQL_PASSWORD = "password"
SQL_IP = "35.193.140.4"
PORT = 5432
DATABASE = "postgres"

In [3]:
connection_string = f"postgresql+psycopg2://{SQL_USERNAME}:{SQL_PASSWORD}@{SQL_IP}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

# Rotten Tomatoes

In [4]:
df = pd.read_csv("rottentomatoes_2020.csv")
df.head(10)

,Rank,RatingTomatometer,Title,No. of Reviews
0,1.0,98%,Portrait of a Lady on Fire (Portrait de la jeu...,303
1,2.0,89%,1917 (2020),448
2,3.0,91%,The Invisible Man (2020),392
3,4.0,99%,Never Rarely Sometimes Always (2020),203
4,5.0,98%,Hamilton (2020),183
5,6.0,92%,Da 5 Bloods (2020),282
6,7.0,97%,Sound of Metal (2020),175
7,8.0,96%,Nomadland (2020),185
8,9.0,99%,Miss Juneteenth (2020),120
9,10.0,88%,Onward (2020),324


## Clean

In [5]:
df["Title"] = [x.split("(")[0].strip() for x in df.Title]
df.head()

,Rank,RatingTomatometer,Title,No. of Reviews
0,1.0,98%,Portrait of a Lady on Fire,303
1,2.0,89%,1917,448
2,3.0,91%,The Invisible Man,392
3,4.0,99%,Never Rarely Sometimes Always,203
4,5.0,98%,Hamilton,183


In [6]:
df_tomatoes= df.rename(columns={'Rank': 'rank', 'No. of Reviews': 'num_reviews', 'RatingTomatometer': 'rating', 'Title': 'title' })
df_tomatoes['rating'] = [x.strip("%") for x in df_tomatoes.rating]
df_tomatoes.head()

,rank,rating,title,num_reviews
0,1.0,98,Portrait of a Lady on Fire,303
1,2.0,89,1917,448
2,3.0,91,The Invisible Man,392
3,4.0,99,Never Rarely Sometimes Always,203
4,5.0,98,Hamilton,183


In [7]:
df_tomatoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   rank         100 non-null    float64
 1   rating       100 non-null    object 
 2   title        100 non-null    object 
 3   num_reviews  100 non-null    int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 3.2+ KB


In [8]:
df_tomatoes['rank'] = df_tomatoes['rank'].astype(int)
df_tomatoes['rating'] = df_tomatoes['rating'].astype(int)
df_tomatoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   rank         100 non-null    int64 
 1   rating       100 non-null    int64 
 2   title        100 non-null    object
 3   num_reviews  100 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 3.2+ KB


In [9]:
df_tomatoes.head()

,rank,rating,title,num_reviews
0,1,98,Portrait of a Lady on Fire,303
1,2,89,1917,448
2,3,91,The Invisible Man,392
3,4,99,Never Rarely Sometimes Always,203
4,5,98,Hamilton,183


# OMDB

In [10]:
url = f"http://www.omdbapi.com/?apikey={api_key}&t="
top_100 = df["Title"].tolist()

In [11]:
# initialize lists

titles = []
directors = []
actors = []
writers = []
ratings = []
years = []
languages = []
countries =[]
genres = []
awards = []
plots = []
posters = []

# request loop
for movie in top_100:
    
    #make request
    url_movie = url + movie.lower()
    response = requests.get(url_movie)
    
    #check status
    if response.status_code == 200:
        data = response.json()
        
        try:

            #parsing the response
            title = data["Title"]
            director = data["Director"]
            actor = data["Actors"]
            writer = data["Writer"]
            rating = data['Rated']
            year = data["Year"]
            language = data["Language"]
            country = data["Country"]
            genre = data["Genre"]
            award = data["Awards"]
            plot = data["Plot"]
            poster = data["Poster"]

            #append to lists
            titles.append(title)
            directors.append(director)
            actors.append(actor)
            writers.append(writer)
            ratings.append(rating)
            years.append(year)
            languages.append(language)
            countries.append(country)
            genres.append(genre)
            awards.append(award)
            plots.append(plot)
            posters.append(poster)

        except (KeyError, IndexError):
            print("Missing field/result... skipping.") 
            titles.append(np.nan)
            directors.append(np.nan)
            actors.append(np.nan)
            writers.append(np.nan)
            ratings.append(np.nan)
            years.append(np.nan)
            languages.append(np.nan)
            countries.append(np.nan)
            genres.append(np.nan)
            awards.append(np.nan)
            plots.append(np.nan)
            posters.append(np.nan)

    else:
        print({response.status_code})
        
# make the dataframe
df_movie = pd.DataFrame()
df_movie["Title"] = titles
df_movie["Director"] = directors
df_movie["Actors"] = actors
df_movie["Writer"] = writers
df_movie["Rated"] = ratings
df_movie["Year"] = years
df_movie["Language"] = languages
df_movie["Country"] = countries
df_movie["Genre"] = genres
df_movie["Awards"] = awards
df_movie["Plot"] = plots
df_movie["Poster"] = posters

#display
df_movie.head(10)

Missing field/result... skipping.
Missing field/result... skipping.


,Title,Director,Actors,Writer,Rated,Year,Language,Country,Genre,Awards,Plot,Poster
0,Portrait of a Lady on Fire,Céline Sciamma,"Noémie Merlant, Adèle Haenel, Luàna Bajrami, V...",Céline Sciamma,R,2019,"French, Italian",France,"Drama, Romance",Nominated for 1 Golden Globe. Another 44 wins ...,On an isolated island in Brittany at the end o...,https://m.media-amazon.com/images/M/MV5BNjgwNj...
1,1917,Sam Mendes,"Dean-Charles Chapman, George MacKay, Daniel Ma...","Sam Mendes, Krysty Wilson-Cairns",R,2019,"English, French, German","USA, UK, India, Spain, Canada, China","Drama, War",Won 3 Oscars. Another 111 wins & 171 nominations.,"April 6th, 1917. As a regiment assembles to wa...",https://m.media-amazon.com/images/M/MV5BOTdmNT...
2,The Invisible Man,Leigh Whannell,"Elisabeth Moss, Oliver Jackson-Cohen, Harriet ...","Leigh Whannell (screenplay), Leigh Whannell (s...",R,2020,English,"Canada, Australia, USA","Drama, Horror, Mystery, Sci-Fi, Thriller",5 wins & 12 nominations.,When Cecilia's abusive ex takes his own life a...,https://m.media-amazon.com/images/M/MV5BZjFhM2...
3,Never Rarely Sometimes Always,Eliza Hittman,"Eliazar Jimenez, David Buneta, Christian Cleme...",Eliza Hittman,PG-13,2020,English,"USA, UK",Drama,6 wins & 16 nominations.,A pair of teenage girls in rural Pennsylvania ...,https://m.media-amazon.com/images/M/MV5BOGUzZj...
4,Hamilton,Thomas Kail,"Daveed Diggs, Renée Elise Goldsberry, Jonathan...","Lin-Manuel Miranda (book), Ron Chernow (inspir...",PG-13,2020,English,USA,"Biography, Drama, History, Musical",1 win & 2 nominations.,The real life of one of America's foremost fou...,https://m.media-amazon.com/images/M/MV5BNjViNW...
5,Da 5 Bloods,Spike Lee,"Delroy Lindo, Jonathan Majors, Clarke Peters, ...","Danny Bilson, Paul De Meo, Kevin Willmott, Spi...",R,2020,"English, Finnish, French, Vietnamese",USA,"Adventure, Drama, War",4 wins & 3 nominations.,Four African American vets battle the forces o...,https://m.media-amazon.com/images/M/MV5BNjE4OD...
6,Sound of Metal,Darius Marder,"Riz Ahmed, Olivia Cooke, Paul Raci, Mathieu Am...","Darius Marder (screenplay by), Derek Cianfranc...",R,2019,"English, American Sign Language, French","Belgium, USA","Drama, Music",5 wins & 7 nominations.,A heavy-metal drummer's life is thrown into fr...,https://m.media-amazon.com/images/M/MV5BNjcyYj...
7,Nomadland,Chloé Zhao,"Frances McDormand, Gay DeForest, Patricia Grie...","Jessica Bruder (book), Chloé Zhao (screenplay by)",R,2020,English,"USA, Germany",Drama,19 wins & 12 nominations.,After losing everything in the Great Recession...,https://m.media-amazon.com/images/M/MV5BOTljNj...
8,Miss Juneteenth,Channing Godfrey Peoples,"Nicole Beharie, Kendrick Sampson, Alexis Chika...",Channing Godfrey Peoples,Unrated,2020,English,USA,Drama,2 wins & 4 nominations.,A former beauty queen and single mom prepares ...,https://m.media-amazon.com/images/M/MV5BNTA5ZW...
9,Onward,Dan Scanlon,"Tom Holland, Chris Pratt, Julia Louis-Dreyfus,...","Dan Scanlon (original story by), Keith Bunin (...",PG,2020,English,USA,"Animation, Adventure, Comedy, Family, Fantasy",2 wins & 4 nominations.,Two elven brothers embark on a quest to bring ...,https://m.media-amazon.com/images/M/MV5BMTZlYz...


In [12]:
df_movie.columns

Index(['Title', 'Director', 'Actors', 'Writer', 'Rated', 'Year', 'Language',
       'Country', 'Genre', 'Awards', 'Plot', 'Poster'],
      dtype='object')

In [13]:
df_movie= df_movie.rename(columns={'Title': 'title', 'Director': 'director', 'Actors': 'actors', 'Writer': 'writer', 'Rated': 'rated', 'Year': 'year', 'Language': 'language', 'Country': 'country', 'Genre': 'genre', 'Awards': 'awards', 'Plot': 'plot', 'Poster': 'poster' })
df_movie.head()

,title,director,actors,writer,rated,year,language,country,genre,awards,plot,poster
0,Portrait of a Lady on Fire,Céline Sciamma,"Noémie Merlant, Adèle Haenel, Luàna Bajrami, V...",Céline Sciamma,R,2019,"French, Italian",France,"Drama, Romance",Nominated for 1 Golden Globe. Another 44 wins ...,On an isolated island in Brittany at the end o...,https://m.media-amazon.com/images/M/MV5BNjgwNj...
1,1917,Sam Mendes,"Dean-Charles Chapman, George MacKay, Daniel Ma...","Sam Mendes, Krysty Wilson-Cairns",R,2019,"English, French, German","USA, UK, India, Spain, Canada, China","Drama, War",Won 3 Oscars. Another 111 wins & 171 nominations.,"April 6th, 1917. As a regiment assembles to wa...",https://m.media-amazon.com/images/M/MV5BOTdmNT...
2,The Invisible Man,Leigh Whannell,"Elisabeth Moss, Oliver Jackson-Cohen, Harriet ...","Leigh Whannell (screenplay), Leigh Whannell (s...",R,2020,English,"Canada, Australia, USA","Drama, Horror, Mystery, Sci-Fi, Thriller",5 wins & 12 nominations.,When Cecilia's abusive ex takes his own life a...,https://m.media-amazon.com/images/M/MV5BZjFhM2...
3,Never Rarely Sometimes Always,Eliza Hittman,"Eliazar Jimenez, David Buneta, Christian Cleme...",Eliza Hittman,PG-13,2020,English,"USA, UK",Drama,6 wins & 16 nominations.,A pair of teenage girls in rural Pennsylvania ...,https://m.media-amazon.com/images/M/MV5BOGUzZj...
4,Hamilton,Thomas Kail,"Daveed Diggs, Renée Elise Goldsberry, Jonathan...","Lin-Manuel Miranda (book), Ron Chernow (inspir...",PG-13,2020,English,USA,"Biography, Drama, History, Musical",1 win & 2 nominations.,The real life of one of America's foremost fou...,https://m.media-amazon.com/images/M/MV5BNjViNW...


In [14]:
df_movie.to_csv("omdb.csv")

In [15]:
df_movie_sub = df_movie.loc[ :, ['title', 'writer', 'awards', 'year', 'plot', 'poster']]
df_movie_sub

,title,writer,awards,year,plot,poster
0,Portrait of a Lady on Fire,Céline Sciamma,Nominated for 1 Golden Globe. Another 44 wins ...,2019,On an isolated island in Brittany at the end o...,https://m.media-amazon.com/images/M/MV5BNjgwNj...
1,1917,"Sam Mendes, Krysty Wilson-Cairns",Won 3 Oscars. Another 111 wins & 171 nominations.,2019,"April 6th, 1917. As a regiment assembles to wa...",https://m.media-amazon.com/images/M/MV5BOTdmNT...
2,The Invisible Man,"Leigh Whannell (screenplay), Leigh Whannell (s...",5 wins & 12 nominations.,2020,When Cecilia's abusive ex takes his own life a...,https://m.media-amazon.com/images/M/MV5BZjFhM2...
3,Never Rarely Sometimes Always,Eliza Hittman,6 wins & 16 nominations.,2020,A pair of teenage girls in rural Pennsylvania ...,https://m.media-amazon.com/images/M/MV5BOGUzZj...
4,Hamilton,"Lin-Manuel Miranda (book), Ron Chernow (inspir...",1 win & 2 nominations.,2020,The real life of one of America's foremost fou...,https://m.media-amazon.com/images/M/MV5BNjViNW...
...,...,...,...,...,...,...
95,Les Misérables,"William Nicholson (screenplay by), Alain Boubl...",Won 3 Oscars. Another 83 wins & 175 nominations.,2012,"In 19th-century France, Jean Valjean, who for ...",https://m.media-amazon.com/images/M/MV5BMTQ4ND...
96,Calm with Horses,"Colin Barrett (based on the short story by), J...",1 win & 6 nominations.,2019,Douglas 'Arm' Armstrong has become the feared ...,https://m.media-amazon.com/images/M/MV5BODcyMD...
97,And Then We Danced,Levan Akin,24 wins & 21 nominations.,2019,A passionate coming-of-age tale set amidst the...,https://m.media-amazon.com/images/M/MV5BMDJhYz...
98,My Darling Vivian,N/A,1 win & 3 nominations.,2020,"The story of Vivian Liberto, Johnny Cash's fir...",https://m.media-amazon.com/images/M/MV5BZmJlNz...


In [16]:
df_movie_sub.to_csv(output_data_file)

# Genre

In [85]:
genres = [] 
for index, row in df_movie.iterrows():
    if not pd.isnull(row.genre):
        genres.extend(row.genre.strip().split(", "))
    
genres = set(genres)
df_genre = pd.DataFrame()
df_genre["genre"] = list(genres)
df_genre

,genre
0,Horror
1,Sci-Fi
2,Crime
3,Fantasy
4,War
5,Animation
6,Family
7,Thriller
8,Documentary
9,Action


In [86]:
#conn = engine.connect()
#df_genre.to_sql("genres", schema="public", con=conn, if_exists="append", index=False, method="multi")
#conn.close()

# Language

In [103]:
languages = [] 
for index, row in df_movie.iterrows():
    if not pd.isnull(row.language):
        languages.extend(row.language.split(", "))
    
languages = set(languages)
df_language = pd.DataFrame()
df_language["language"] = list(languages)
df_language

,language
0,Italian
1,Bulgarian
2,Cantonese
3,Welsh
4,N/A
5,Chinese
6,Portuguese
7,Spanish
8,Hebrew
9,Finnish


In [104]:
#conn = engine.connect()
#df_language.to_sql("languages", schema="public", con=conn, if_exists="append", index=False, method="multi")
#conn.close()

# Actor

In [112]:
actors = [] 
for index, row in df_movie.iterrows():
    if not pd.isnull(row.actors):
        actors.extend(row.actors.split(", "))
    
actors = set(actors)
df_actor = pd.DataFrame()
df_actor["actor_name"] = list(actors)
df_actor

,actor_name
0,Adam Horovitz
1,Ahmad Nicholas Ferguson
2,Hsiao-Yuan Ha
3,Daveed Diggs
4,Robbie Jones
...,...
371,Gustavo Di Dalva
372,Ashley Bell
373,Ana Hoffman
374,Burak Agan


In [113]:
#conn = engine.connect()
#df_actor.to_sql("actors", schema="public", con=conn, if_exists="append", index=False, method="multi")
#conn.close()

# Country

In [94]:
countries= [] 
for index, row in df_movie.iterrows():
    if not pd.isnull(row.country):
        countries.extend(row.country.split(", "))
    
countries = set(countries)
df_country = pd.DataFrame()
df_country["country"] = list(countries)
df_country

,country
0,Ireland
1,Spain
2,Italy
3,Belgium
4,Hong Kong
5,France
6,Georgia
7,Sweden
8,USA
9,Australia


In [95]:
#conn = engine.connect()
#df_country.to_sql("countries", schema="public", con=conn, if_exists="append", index=False, method="multi")
#conn.close()

# Director

In [123]:
directors = [] 
for index, row in df_movie.iterrows():
    if not pd.isnull(row.director):
        directors.extend(row.director.split(", "))
    
directors = set(directors)
df_director = pd.DataFrame()
df_director["director_name"] = list(directors)
df_director

,director_name
0,Autumn de Wilde
1,Enda Loughman
2,Ekwa Msangi
3,Arthur Jones
4,Hao Wu
...,...
117,Elyse Steinberg
118,Danielle Krudy
119,Regina King
120,Kitty Green


In [124]:
#conn = engine.connect()
#df_director.to_sql("directors", schema="public", con=conn, if_exists="append", index=False, method="multi")
#conn.close()

# Movies

In [27]:
titles = [] 
for index, row in df_movie.iterrows():
    if not pd.isnull(row.title):
        titles.extend(row.title.split(","))
    
titles = set(titles)
df_title = pd.DataFrame()
df_title["title"] = list(titles)
df_title

,title
0,76 Days
1,The Swerve
2,Athlete A
3,Onward
4,Fourteen
...,...
96,The Painter and the Thief
97,First Cow
98,Disclosure
99,Ordinary Love


In [28]:
df_movie_sub.head()

,title,writer,awards,year,plot,poster
0,Portrait of a Lady on Fire,Céline Sciamma,Nominated for 1 Golden Globe. Another 44 wins ...,2019,On an isolated island in Brittany at the end o...,https://m.media-amazon.com/images/M/MV5BNjgwNj...
1,1917,"Sam Mendes, Krysty Wilson-Cairns",Won 3 Oscars. Another 111 wins & 171 nominations.,2019,"April 6th, 1917. As a regiment assembles to wa...",https://m.media-amazon.com/images/M/MV5BOTdmNT...
2,The Invisible Man,"Leigh Whannell (screenplay), Leigh Whannell (s...",5 wins & 12 nominations.,2020,When Cecilia's abusive ex takes his own life a...,https://m.media-amazon.com/images/M/MV5BZjFhM2...
3,Never Rarely Sometimes Always,Eliza Hittman,6 wins & 16 nominations.,2020,A pair of teenage girls in rural Pennsylvania ...,https://m.media-amazon.com/images/M/MV5BOGUzZj...
4,Hamilton,"Lin-Manuel Miranda (book), Ron Chernow (inspir...",1 win & 2 nominations.,2020,The real life of one of America's foremost fou...,https://m.media-amazon.com/images/M/MV5BNjViNW...


In [29]:
df_movie_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   98 non-null     object
 1   writer  98 non-null     object
 2   awards  98 non-null     object
 3   year    98 non-null     object
 4   plot    98 non-null     object
 5   poster  98 non-null     object
dtypes: object(6)
memory usage: 4.8+ KB


In [30]:
#conn = engine.connect()
#df_movie_sub.to_sql("movies", schema="public", con=conn, if_exists="append", index=False, method="multi")
#conn.close()

# Rating

In [31]:
query = """
        SELECT film_id, title
        FROM movies
        """

In [32]:
conn = engine.connect()
df_film_id = pd.read_sql(query,con=conn)
conn.close()

df_film_id.head()

,film_id,title
0,1,Portrait of a Lady on Fire
1,2,1917
2,3,The Invisible Man
3,4,Never Rarely Sometimes Always
4,5,Hamilton


In [33]:
df_tomatoes.head()

,rank,rating,title,num_reviews
0,1,98,Portrait of a Lady on Fire,303
1,2,89,1917,448
2,3,91,The Invisible Man,392
3,4,99,Never Rarely Sometimes Always,203
4,5,98,Hamilton,183


In [34]:
df_combo = df_tomatoes.join(df_film_id.set_index('title'), on='title')
df_combo.head()

,rank,rating,title,num_reviews,film_id
0,1,98,Portrait of a Lady on Fire,303,1.0
1,2,89,1917,448,2.0
2,3,91,The Invisible Man,392,3.0
3,4,99,Never Rarely Sometimes Always,203,4.0
4,5,98,Hamilton,183,5.0


In [35]:
df_combo_sub = df_combo.loc[ :, ['rank', 'rating', 'num_reviews', 'film_id']]
df_combo_sub.head()

,rank,rating,num_reviews,film_id
0,1,98,303,1.0
1,2,89,448,2.0
2,3,91,392,3.0
3,4,99,203,4.0
4,5,98,183,5.0


In [36]:
#conn = engine.connect()
#df_combo_sub.to_sql("ratings", schema="public", con=conn, if_exists="append", index=False, method="multi")
#conn.close()

# film_actor

In [114]:
query = """
        SELECT * 
        FROM actors
        """

In [115]:
conn = engine.connect()
df_actor_id = pd.read_sql(query,con=conn)
conn.close()

df_actor_id.head()

,actor_id,actor_name,last_updated
0,1,Adam Horovitz,2020-12-19 16:24:23.495214
1,2,Ahmad Nicholas Ferguson,2020-12-19 16:24:23.495214
2,3,Hsiao-Yuan Ha,2020-12-19 16:24:23.495214
3,4,Daveed Diggs,2020-12-19 16:24:23.495214
4,5,Robbie Jones,2020-12-19 16:24:23.495214


In [116]:
omdb_df = pd.read_csv("omdb.csv")
omdb_df = omdb_df.loc[:, ['title', 'director', 'actors', 'rated', 'year', 'language', 'country', 'genre', 'awards', 'plot', 'poster']]
omdb_df.head()

,title,director,actors,rated,year,language,country,genre,awards,plot,poster
0,Portrait of a Lady on Fire,Céline Sciamma,"Noémie Merlant, Adèle Haenel, Luàna Bajrami, V...",R,2019,"French, Italian",France,"Drama, Romance",Nominated for 1 Golden Globe. Another 44 wins ...,On an isolated island in Brittany at the end o...,https://m.media-amazon.com/images/M/MV5BNjgwNj...
1,1917,Sam Mendes,"Dean-Charles Chapman, George MacKay, Daniel Ma...",R,2019,"English, French, German","USA, UK, India, Spain, Canada, China","Drama, War",Won 3 Oscars. Another 111 wins & 171 nominations.,"April 6th, 1917. As a regiment assembles to wa...",https://m.media-amazon.com/images/M/MV5BOTdmNT...
2,The Invisible Man,Leigh Whannell,"Elisabeth Moss, Oliver Jackson-Cohen, Harriet ...",R,2020,English,"Canada, Australia, USA","Drama, Horror, Mystery, Sci-Fi, Thriller",5 wins & 12 nominations.,When Cecilia's abusive ex takes his own life a...,https://m.media-amazon.com/images/M/MV5BZjFhM2...
3,Never Rarely Sometimes Always,Eliza Hittman,"Eliazar Jimenez, David Buneta, Christian Cleme...",PG-13,2020,English,"USA, UK",Drama,6 wins & 16 nominations.,A pair of teenage girls in rural Pennsylvania ...,https://m.media-amazon.com/images/M/MV5BOGUzZj...
4,Hamilton,Thomas Kail,"Daveed Diggs, Renée Elise Goldsberry, Jonathan...",PG-13,2020,English,USA,"Biography, Drama, History, Musical",1 win & 2 nominations.,The real life of one of America's foremost fou...,https://m.media-amazon.com/images/M/MV5BNjViNW...


In [117]:
film_actors = [] # this will be a list of dictionaries

for indx, row in omdb_df.iterrows():
    title = row.title
    if not pd.isnull(row.actors):
        actors = row.actors.split(",") # this will be a list
    for actor in actors: #create dict and append
        film_actor = {"title": title, "actor_name": actor}
        film_actors.append(film_actor)
        
film_actors_df = pd.DataFrame(film_actors)
film_actors_df

,title,actor_name
0,Portrait of a Lady on Fire,Noémie Merlant
1,Portrait of a Lady on Fire,Adèle Haenel
2,Portrait of a Lady on Fire,Luàna Bajrami
3,Portrait of a Lady on Fire,Valeria Golino
4,1917,Dean-Charles Chapman
...,...,...
388,My Darling Vivian,Ginnifer Goodwin
389,Dinner in America,Kyle Gallner
390,Dinner in America,Emily Skeggs
391,Dinner in America,Mary Anderson


In [118]:
df_actor_id.head()

,actor_id,actor_name,last_updated
0,1,Adam Horovitz,2020-12-19 16:24:23.495214
1,2,Ahmad Nicholas Ferguson,2020-12-19 16:24:23.495214
2,3,Hsiao-Yuan Ha,2020-12-19 16:24:23.495214
3,4,Daveed Diggs,2020-12-19 16:24:23.495214
4,5,Robbie Jones,2020-12-19 16:24:23.495214


In [119]:
newDF = film_actors_df.merge(df_actor_id, on ="actor_name")
newDF.head()

,title,actor_name,actor_id,last_updated
0,Portrait of a Lady on Fire,Noémie Merlant,225,2020-12-19 16:24:23.495214
1,1917,Dean-Charles Chapman,258,2020-12-19 16:24:23.495214
2,The Invisible Man,Elisabeth Moss,38,2020-12-19 16:24:23.495214
3,Shirley,Elisabeth Moss,38,2020-12-19 16:24:23.495214
4,Never Rarely Sometimes Always,Eliazar Jimenez,134,2020-12-19 16:24:23.495214


In [120]:
newnewDF = newDF.merge(df_film_id, on="title")
newnewDF.head()

,title,actor_name,actor_id,last_updated,film_id
0,Portrait of a Lady on Fire,Noémie Merlant,225,2020-12-19 16:24:23.495214,1
1,1917,Dean-Charles Chapman,258,2020-12-19 16:24:23.495214,2
2,The Invisible Man,Elisabeth Moss,38,2020-12-19 16:24:23.495214,3
3,Shirley,Elisabeth Moss,38,2020-12-19 16:24:23.495214,81
4,Never Rarely Sometimes Always,Eliazar Jimenez,134,2020-12-19 16:24:23.495214,4


In [121]:
film_actors = newnewDF.drop(columns=['title', 'actor_name', 'last_updated'])
film_actors.head()

,actor_id,film_id
0,225,1
1,258,2
2,38,3
3,38,81
4,134,4


In [122]:
#conn = engine.connect()
#film_actors.to_sql("film_actors", schema="public", con=conn, if_exists="append", index=False, method="multi")
#conn.close()

# film_countries

In [96]:
query = """
        SELECT * 
        FROM countries
        """

In [97]:
conn = engine.connect()
df_countries = pd.read_sql(query,con=conn)
conn.close()

df_countries.head()

,country_id,country,last_updated
0,1,Ireland,2020-12-19 16:12:22.299979
1,2,Spain,2020-12-19 16:12:22.299979
2,3,Italy,2020-12-19 16:12:22.299979
3,4,Belgium,2020-12-19 16:12:22.299979
4,5,Hong Kong,2020-12-19 16:12:22.299979


In [98]:
film_countries = [] # this will be a list of dictionaries

for indx, row in omdb_df.iterrows():
    title = row.title
    if not pd.isnull(row.actors):
        countries = row.country.split(",") # this will be a list
    for country in countries: #create dict and append
        film_country = {"title": title, "country": country}
        film_countries.append(film_country)
        
film_countries_df = pd.DataFrame(film_countries)
film_countries_df

,title,country
0,Portrait of a Lady on Fire,France
1,1917,USA
2,1917,UK
3,1917,India
4,1917,Spain
...,...,...
141,And Then We Danced,Sweden
142,And Then We Danced,Georgia
143,And Then We Danced,France
144,My Darling Vivian,USA


In [99]:
new_country_title = film_countries_df.merge(df_countries, on="country")
new_country_title.head()

,title,country,country_id,last_updated
0,Portrait of a Lady on Fire,France,6,2020-12-19 16:12:22.299979
1,1917,USA,9,2020-12-19 16:12:22.299979
2,Never Rarely Sometimes Always,USA,9,2020-12-19 16:12:22.299979
3,Hamilton,USA,9,2020-12-19 16:12:22.299979
4,Da 5 Bloods,USA,9,2020-12-19 16:12:22.299979


In [100]:
film_countries = new_country_title.merge(df_film_id, on="title")
film_countries.head()

,title,country,country_id,last_updated,film_id
0,Portrait of a Lady on Fire,France,6,2020-12-19 16:12:22.299979,1
1,1917,USA,9,2020-12-19 16:12:22.299979,2
2,Never Rarely Sometimes Always,USA,9,2020-12-19 16:12:22.299979,4
3,Hamilton,USA,9,2020-12-19 16:12:22.299979,5
4,Da 5 Bloods,USA,9,2020-12-19 16:12:22.299979,6


In [101]:
film_countries = film_countries.drop(columns=['title', 'country', 'last_updated'])
film_countries.head()

,country_id,film_id
0,6,1
1,9,2
2,9,4
3,9,5
4,9,6


In [102]:
#conn = engine.connect()
#film_countries.to_sql("film_countries", schema="public", con=conn, if_exists="append", index=False, method="multi")
#conn.close()

# film_directors

In [125]:
query = """
        SELECT * 
        FROM directors
        """

In [126]:
conn = engine.connect()
df_directors = pd.read_sql(query,con=conn)
conn.close()

df_directors.head()

,director_id,director_name,last_updated
0,1,Autumn de Wilde,2020-12-19 16:27:20.916794
1,2,Enda Loughman,2020-12-19 16:27:20.916794
2,3,Ekwa Msangi,2020-12-19 16:27:20.916794
3,4,Arthur Jones,2020-12-19 16:27:20.916794
4,5,Hao Wu,2020-12-19 16:27:20.916794


In [127]:
film_directors = [] # this will be a list of dictionaries

for indx, row in omdb_df.iterrows():
    title = row.title
    if not pd.isnull(row.director):
        directors = row.director.split(",") # this will be a list
    for director in directors: #create dict and append
        film_director = {"title": title, "director_name": director}
        film_directors.append(film_director)
        
film_directors_df = pd.DataFrame(film_directors)
film_directors_df

,title,director_name
0,Portrait of a Lady on Fire,Céline Sciamma
1,1917,Sam Mendes
2,The Invisible Man,Leigh Whannell
3,Never Rarely Sometimes Always,Eliza Hittman
4,Hamilton,Thomas Kail
...,...,...
121,Les Misérables,Tom Hooper
122,Calm with Horses,Nick Rowland
123,And Then We Danced,Levan Akin
124,My Darling Vivian,Matt Riddlehoover


In [128]:
new_director_title = film_directors_df.merge(df_directors, on="director_name")
new_director_title.head()

,title,director_name,director_id,last_updated
0,Portrait of a Lady on Fire,Céline Sciamma,34,2020-12-19 16:27:20.916794
1,1917,Sam Mendes,73,2020-12-19 16:27:20.916794
2,The Invisible Man,Leigh Whannell,35,2020-12-19 16:27:20.916794
3,Never Rarely Sometimes Always,Eliza Hittman,102,2020-12-19 16:27:20.916794
4,Hamilton,Thomas Kail,20,2020-12-19 16:27:20.916794


In [129]:
film_directors = new_director_title.merge(df_film_id, on="title")
film_directors.head()

,title,director_name,director_id,last_updated,film_id
0,Portrait of a Lady on Fire,Céline Sciamma,34,2020-12-19 16:27:20.916794,1
1,1917,Sam Mendes,73,2020-12-19 16:27:20.916794,2
2,The Invisible Man,Leigh Whannell,35,2020-12-19 16:27:20.916794,3
3,Never Rarely Sometimes Always,Eliza Hittman,102,2020-12-19 16:27:20.916794,4
4,Hamilton,Thomas Kail,20,2020-12-19 16:27:20.916794,5


In [130]:
film_directors = film_directors.drop(columns=['title', 'director_name', 'last_updated'])
film_directors.head()

,director_id,film_id
0,34,1
1,73,2
2,35,3
3,102,4
4,20,5


In [131]:
#conn = engine.connect()
#film_directors.to_sql("film_directors", schema="public", con=conn, if_exists="append", index=False, method="multi")
#conn.close()

# film_genres

In [87]:
query = """
        SELECT * 
        FROM genres
        """

In [88]:
conn = engine.connect()
df_genres = pd.read_sql(query,con=conn)
conn.close()

df_genres.head()

,genre_id,genre,last_updated
0,1,Horror,2020-12-19 16:06:29.000084
1,2,Sci-Fi,2020-12-19 16:06:29.000084
2,3,Crime,2020-12-19 16:06:29.000084
3,4,Fantasy,2020-12-19 16:06:29.000084
4,5,War,2020-12-19 16:06:29.000084


In [89]:
film_genres = [] # this will be a list of dictionaries

for indx, row in omdb_df.iterrows():
    title = row.title
    if not pd.isnull(row.genre):
        genres = row.genre.split(",") # this will be a list
    for genre in genres: #create dict and append
        film_genre = {"title": title, "genre": genre}
        film_genres.append(film_genre)
        
film_genres_df = pd.DataFrame(film_genres)
film_genres_df

,title,genre
0,Portrait of a Lady on Fire,Drama
1,Portrait of a Lady on Fire,Romance
2,1917,Drama
3,1917,War
4,The Invisible Man,Drama
...,...,...
231,My Darling Vivian,Documentary
232,My Darling Vivian,Biography
233,Dinner in America,Comedy
234,Dinner in America,Drama


In [90]:
new_genre_title = film_genres_df.merge(df_genres, on="genre")
new_genre_title.head()

,title,genre,genre_id,last_updated
0,Portrait of a Lady on Fire,Drama,14,2020-12-19 16:06:29.000084
1,1917,Drama,14,2020-12-19 16:06:29.000084
2,The Invisible Man,Drama,14,2020-12-19 16:06:29.000084
3,Never Rarely Sometimes Always,Drama,14,2020-12-19 16:06:29.000084
4,Sound of Metal,Drama,14,2020-12-19 16:06:29.000084


In [91]:
film_genres = new_genre_title.merge(df_film_id, on="title")
film_genres.head()

,title,genre,genre_id,last_updated,film_id
0,Portrait of a Lady on Fire,Drama,14,2020-12-19 16:06:29.000084,1
1,1917,Drama,14,2020-12-19 16:06:29.000084,2
2,The Invisible Man,Drama,14,2020-12-19 16:06:29.000084,3
3,Never Rarely Sometimes Always,Drama,14,2020-12-19 16:06:29.000084,4
4,Sound of Metal,Drama,14,2020-12-19 16:06:29.000084,7


In [92]:
film_genres = film_genres.drop(columns=['title', 'genre', 'last_updated'])
film_genres.head()

,genre_id,film_id
0,14,1
1,14,2
2,14,3
3,14,4
4,14,7


In [93]:
#conn = engine.connect()
#film_genres.to_sql("film_genres", schema="public", con=conn, if_exists="append", index=False, method="multi")
#conn.close()

# film_languages

In [105]:
query = """
        SELECT * 
        FROM languages
        """

In [106]:
conn = engine.connect()
df_languages = pd.read_sql(query,con=conn)
conn.close()

df_languages.head()

,language_id,language,last_updated
0,1,Italian,2020-12-19 16:15:37.836598
1,2,Bulgarian,2020-12-19 16:15:37.836598
2,3,Cantonese,2020-12-19 16:15:37.836598
3,4,Welsh,2020-12-19 16:15:37.836598
4,5,N/A,2020-12-19 16:15:37.836598


In [107]:
film_languages = [] # this will be a list of dictionaries

for indx, row in omdb_df.iterrows():
    title = row.title
    if not pd.isnull(row.language):
        languages = row.language.split(",") # this will be a list
    for language in languages: #create dict and append
        film_language = {"title": title, "language": language}
        film_languages.append(film_language)
        
film_languages_df = pd.DataFrame(film_languages)
film_languages_df

,title,language
0,Portrait of a Lady on Fire,French
1,Portrait of a Lady on Fire,Italian
2,1917,English
3,1917,French
4,1917,German
...,...,...
130,Les Misérables,English
131,Calm with Horses,English
132,And Then We Danced,Georgian
133,My Darling Vivian,English


In [108]:
new_language_title = film_languages_df.merge(df_languages, on="language")
new_language_title.head()

,title,language,language_id,last_updated
0,Portrait of a Lady on Fire,French,14,2020-12-19 16:15:37.836598
1,1917,English,12,2020-12-19 16:15:37.836598
2,The Invisible Man,English,12,2020-12-19 16:15:37.836598
3,Never Rarely Sometimes Always,English,12,2020-12-19 16:15:37.836598
4,Hamilton,English,12,2020-12-19 16:15:37.836598


In [109]:
film_languages = new_language_title.merge(df_film_id, on="title")
film_languages.head()

,title,language,language_id,last_updated,film_id
0,Portrait of a Lady on Fire,French,14,2020-12-19 16:15:37.836598,1
1,1917,English,12,2020-12-19 16:15:37.836598,2
2,The Invisible Man,English,12,2020-12-19 16:15:37.836598,3
3,Never Rarely Sometimes Always,English,12,2020-12-19 16:15:37.836598,4
4,Hamilton,English,12,2020-12-19 16:15:37.836598,5


In [110]:
film_languages = film_languages.drop(columns=['title', 'language', 'last_updated'])
film_languages.head()

,language_id,film_id
0,14,1
1,12,2
2,12,3
3,12,4
4,12,5


In [111]:
#conn = engine.connect()
#film_languages.to_sql("film_languages", schema="public", con=conn, if_exists="append", index=False, method="multi")
#conn.close()